In [1]:
# !/usr/bin/python
# -*- coding: utf-8
from __future__ import division, unicode_literals

%matplotlib inline
# http://matplotlib.org/users/pyplot_tutorial.html
import matplotlib.pyplot as plt
import numpy as np
from pandas import Series, DataFrame
import pandas as pd

# http://nbviewer.ipython.org/
import xlsxwriter
from pandas import ExcelWriter
import time
import datetime
from datetime import timedelta
import operator
import re

In [2]:
file_name ="data/user_polished.xlsx"
xls = pd.ExcelFile(file_name)
dt_user_per1 = xls.parse("Per1", header=1) #"data_polished.xlsx"파일의 "Per1"이라는 시트를 가져옴. 1번 row를 헤더로 둠
dt_user_per2 = xls.parse("Per2", header=1) #"data_polished.xlsx"파일의 "Per2"이라는 시트를 가져옴. 1번 row를 헤더로 둠
dt_user_per3 = xls.parse("Per3", header=1) #"data_polished.xlsx"파일의 "Per3"이라는 시트를 가져옴. 1번 row를 헤더로 둠

del dt_user_per1[0] #0이라는 column이 있어서 지움
del dt_user_per2[0]
del dt_user_per3[0]

In [3]:
file_name ="data/activity_raw.xlsx"
xls = pd.ExcelFile(file_name)
dt_activity = xls.parse('Sheet1', header=0)
dt_activity = dt_activity.reindex(dt_activity.index.drop(0))  #"활동명", "마감일" 등 한글로 써져 있는거 지움
dt_activity = dt_activity.reindex(dt_activity.index.drop(1))  # 활동혜택의 세부사항때문에 생기는 두번째 줄도 지움

dt_activity.head(1)

,activity,close,start,end,region,category,s1,s2,s3,i1,...,d1,d2,url,img,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
2,[MIOS] 캄보디아 해외봉사단 5기 모집,2016-04-09 00:00:00,2016-06-23 00:00:00,2016-06-30 00:00:00,NaN,봉사단,NaN,NaN,NaN,여행/호텔/항공,...,미디어/디자인,NaN,http://cafe.naver.com/specup/4122665,http://i.imgur.com/8ujCaUw.jpg?1,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def score_of_a_user_per1(i, dt_user): # 한명의 사용자에 대한 모든 활동의 점수를 구하고 상위 7개를 뽑는 함수
    result = [dt_user['email'][i], dt_user['category'][i], dt_user['start'][i], dt_user['end'][i]
             , dt_user['region'][i]] #해당 사용자에 대한 결과를 모으는 리스트. 최종적으로 이걸 return하여 리스트에 한줄씩 쌓음
    
    list_user_region = dt_user['region'][i].split(", ") # 해당 사용자가 활동 가능한 지역
    user_start = dt_user['start'][i] # 해당 사용자가 활동 가능한 기간 "시작일"
    user_end = dt_user['end'][i] # 해당 사용자가 활동 가능한 기간 "종료일"
    
    list_user_category = [] # 해당 사용자가 원하는 "카테고리"를 담아두는 리스트
    if(type(dt_user['category'][i])!=float):
        list_user_category = dt_user['category'][i].split(", ") 
    
    list_interest = [] #해당 사용자의 "관심사"를 담아두는 리스트 
    for k in range(1, 17):
        if(dt_user['i'+str(k)][i]=="-"): break
        else : list_interest.append(dt_user['i'+str(k)][i])
    
    dic_score = {} #점수를 계산해서 저장해두는 dictionary. 추후 소팅하여 상위 7개를 뽑아낸다. 
    dic_benefit = {} #해당 항목이 사용자와 
    dic_interest = {}
    for j in range(2, dt_activity.index.size+1):
        activity_start = dt_activity['start'][j]
        activity_end = dt_activity['end'][j]
        activity_region = dt_activity['region'][j]
        
        if not (user_start<=activity_start and user_end>=activity_end) : continue # 해당 사용자 활동 가능 기간과 안맞으면 제외.
        if not (type(activity_region)==float): 
            if not ("전국 다 가능" in list_user_region):
                list_activity_region = activity_region.split(", ")
                for a_r in list_activity_region :
                    if a_r in list_user_region : break
                else:
                    continue
        if (dt_activity['category'][j] not in list_user_category): continue # 해당 사용자가 원하는 카테고리에 없으면 아예 제외한다. 
        b_score = 0
        b_match = []
        i_score = 0
        i_match = []
        p1_score = 0
        if dt_activity['b1'][j] =="o" and dt_user['b1'][i]=="O": b_score += 20; b_match.append("b1") 
        if dt_activity['b2'][j] =="o" and dt_user['b2'][i]=="O": b_score += 20; b_match.append("b2") 
        if dt_activity['b3'][j] =="o" and dt_user['b3'][i]=="O": b_score += 10; b_match.append("b3") 
        if dt_activity['b4'][j] =="o" and dt_user['b4'][i]=="O": b_score += 10; b_match.append("b4") 
        if dt_activity['b5'][j] =="o" and dt_user['b5'][i]=="O": b_score += 20; b_match.append("b5") 
        if dt_activity['b6'][j] =="o" and dt_user['b6'][i]=="O": b_score += 20; b_match.append("b6") 
        if dt_activity['b7'][j] =="o" and dt_user['b7'][i]=="O": b_score += 10; b_match.append("b7") 
        if dt_activity['b8'][j] =="o" and dt_user['b8'][i]=="O": b_score += 10; b_match.append("b8") 
        if dt_activity['b9'][j] =="o" and dt_user['b9'][i]=="O": b_score += 10; b_match.append("b9") 
        
        if dt_activity['i1'][j] in list_interest : i_score += 15; i_match.append(dt_activity['i1'][j])
        if dt_activity['i2'][j] in list_interest : i_score += 15; i_match.append(dt_activity['i2'][j])

        p1_score = b_score + i_score
        dic_score[dt_activity['activity'][j]] = p1_score
        if len(b_match)==0 : b_match.append("-")
        if len(i_match)==0 : i_match.append("-")
        dic_benefit[dt_activity['activity'][j]] = b_match
        dic_interest[dt_activity['activity'][j]] = i_match
    sorted_dic_score = sorted(dic_score.items(), key=operator.itemgetter(1), reverse=True)
    
    for n in range(0, 7):        
        if(n<len(sorted_dic_score)):
            if(sorted_dic_score[n][1]>0):
                result.append(sorted_dic_score[n][0])
                result.append(sorted_dic_score[n][1])
                result.append(", ".join(dic_benefit[sorted_dic_score[n][0]]))
                result.append(", ".join(dic_interest[sorted_dic_score[n][0]]))
                result.append("-")
            else :
                result.append("-")
                result.append("-")
                result.append("-")
                result.append("-")
                result.append("-")
            
        else:
            result.append("-")
            result.append("-")
            result.append("-")
            result.append("-")
            result.append("-")
            
    return result


In [5]:
def score_of_a_user_per2(i, dt_user): # 한명의 사용자에 대한 모든 활동의 점수를 구하고 상위 7개를 뽑는 함수
    result = [dt_user['email'][i], dt_user['category'][i], dt_user['start'][i], dt_user['end'][i]
             , dt_user['region'][i]] #해당 사용자에 대한 결과를 모으는 리스트. 최종적으로 이걸 return하여 리스트에 한줄씩 쌓음
    
    user_start = dt_user['start'][i] # 해당 사용자가 활동 가능한 기간 "시작일"
    user_end = dt_user['end'][i] # 해당 사용자가 활동 가능한 기간 "종료일"
    list_user_region = dt_user['region'][i].split(", ") # 해당 사용자가 활동 가능한 지역
    list_user_category = [] # 해당 사용자가 원하는 "카테고리"를 담아두는 리스트
    list_industry = [] # 해당 사용자가 원하는 "산업군"을 담아두는 리스트
    list_interest = [] #해당 사용자의 "관심사"를 담아두는 리스트
    if(type(dt_user['category'][i])!=float):
        list_user_category = dt_user['category'][i].split(", ")    
    
    for k in range(1, 11):
        if(dt_user['d'+str(k)][i]=="-"): break
        else : list_industry.append(dt_user['d'+str(k)][i]) #해당 사용자 산업군 넣음.
     
    for k in range(1, 17):
        if(dt_user['i'+str(k)][i]=="-"): break
        else : list_interest.append(dt_user['i'+str(k)][i])
    
    dic_score = {} #점수를 계산해서 저장해두는 dictionary. 추후 소팅하여 상위 7개를 뽑아낸다. 
    dic_benefit = {} #해당 항목이 사용자와 
    dic_interest = {}
    dic_industry = {}#해당 산업군이 사용자와 겹치는지
    for j in range(2, dt_activity.index.size+1):
        activity_start = dt_activity['start'][j]
        activity_end = dt_activity['end'][j]
        activity_region = dt_activity['region'][j]
        if not (user_start<=activity_start and user_end>=activity_end) : continue # 해당 사용자 활동 가능 기간과 안맞으면 제외.
        if not (type(activity_region)==float): 
            if not ("전국 다 가능" in list_user_region):
                list_activity_region = activity_region.split(", ")
                for a_r in list_activity_region :
                    if a_r in list_user_region : break
                else:
                    continue
        if (dt_activity['category'][j] not in list_user_category): continue # 해당 사용자가 원하는 카테고리에 없으면 아예 제외한다. 
        b_score = 0
        b_match = []
        i_score = 0
        i_match = []
        d_score = 0
        d_match = []
        p2_score = 0
        if dt_activity['b1'][j] =="o" and dt_user['b1'][i]=="O": b_score += 10; b_match.append("b1") 
        if dt_activity['b2'][j] =="o" and dt_user['b2'][i]=="O": b_score += 10; b_match.append("b2") 
        if dt_activity['b3'][j] =="o" and dt_user['b3'][i]=="O": b_score += 15; b_match.append("b3") 
        if dt_activity['b4'][j] =="o" and dt_user['b4'][i]=="O": b_score += 15; b_match.append("b4") 
        if dt_activity['b5'][j] =="o" and dt_user['b5'][i]=="O": b_score += 10; b_match.append("b5") 
        if dt_activity['b6'][j] =="o" and dt_user['b6'][i]=="O": b_score += 10; b_match.append("b6") 
        if dt_activity['b7'][j] =="o" and dt_user['b7'][i]=="O": b_score += 15; b_match.append("b7") 
        if dt_activity['b8'][j] =="o" and dt_user['b8'][i]=="O": b_score += 15; b_match.append("b8") 
        if dt_activity['b9'][j] =="o" and dt_user['b9'][i]=="O": b_score += 15; b_match.append("b9") 
        
        if dt_activity['i1'][j] in list_interest : i_score += 20; i_match.append(dt_activity['i1'][j])
        if dt_activity['i2'][j] in list_interest : i_score += 20; i_match.append(dt_activity['i2'][j])
        
        if dt_activity['d1'][j] in list_industry : d_score += 15; d_match.append(dt_activity['d1'][j])
        if dt_activity['d2'][j] in list_industry : d_score += 15; d_match.append(dt_activity['d2'][j])
        
        
        p2_score = b_score + i_score + d_score
        dic_score[dt_activity['activity'][j]] = p2_score
        if len(b_match)==0 : b_match.append("-")
        if len(i_match)==0 : i_match.append("-")
        if len(d_match)==0 : d_match.append("-")

        dic_benefit[dt_activity['activity'][j]] = b_match
        dic_interest[dt_activity['activity'][j]] = i_match
        dic_industry[dt_activity['activity'][j]] = d_match
    sorted_dic_score = sorted(dic_score.items(), key=operator.itemgetter(1), reverse=True)
    
    for n in range(0, 7):
        if(n<len(sorted_dic_score)):
            if(sorted_dic_score[n][1]>0):
                result.append(sorted_dic_score[n][0])
                result.append(sorted_dic_score[n][1])
                result.append(", ".join(dic_benefit[sorted_dic_score[n][0]]))
                result.append(", ".join(dic_interest[sorted_dic_score[n][0]]))
                result.append(", ".join(dic_industry[sorted_dic_score[n][0]]))
            else :
                result.append("-")
                result.append("-")
                result.append("-")
                result.append("-")
                result.append("-")    
        else:
            result.append("-")
            result.append("-")
            result.append("-")
            result.append("-")
            result.append("-")
    return result

In [6]:
def score_of_a_user_per3(i, dt_user): # 한명의 사용자에 대한 모든 활동의 점수를 구하고 상위 7개를 뽑는 함수
    result = [dt_user['email'][i], dt_user['category'][i], dt_user['start'][i], dt_user['end'][i]
             , dt_user['region'][i]] #해당 사용자에 대한 결과를 모으는 리스트. 최종적으로 이걸 return하여 리스트에 한줄씩 쌓음
    
    user_start = dt_user['start'][i] # 해당 사용자가 활동 가능한 기간 "시작일"
    user_end = dt_user['end'][i] # 해당 사용자가 활동 가능한 기간 "종료일"
    list_user_region = dt_user['region'][i].split(", ") # 해당 사용자가 활동 가능한 지역
    list_user_category = [] # 해당 사용자가 원하는 "카테고리"를 담아두는 리스트
    list_industry = [] # 해당 사용자가 원하는 "산업군"을 담아두는 리스트
    list_interest = [] #해당 사용자의 "관심사"를 담아두는 리스트
    if(type(dt_user['category'][i])!=float):
        list_user_category = dt_user['category'][i].split(", ")    
    
    for k in range(1, 11):
        if(dt_user['d'+str(k)][i]=="-"): break
        else : list_industry.append(dt_user['d'+str(k)][i]) #해당 사용자 산업군 넣음.
     
    for k in range(1, 17):
        if(dt_user['i'+str(k)][i]=="-"): break
        else : list_interest.append(dt_user['i'+str(k)][i])
    
    dic_score = {} #점수를 계산해서 저장해두는 dictionary. 추후 소팅하여 상위 7개를 뽑아낸다. 
    dic_benefit = {} #해당 항목이 사용자와 
    dic_interest = {}
    dic_industry = {}#해당 산업군이 사용자와 겹치는지
    for j in range(2, dt_activity.index.size+1):
        activity_start = dt_activity['start'][j]
        activity_end = dt_activity['end'][j]
        activity_region = dt_activity['region'][j]
        if not (user_start<=activity_start and user_end>=activity_end) : continue # 해당 사용자 활동 가능 기간과 안맞으면 제외.
        if not (type(activity_region)==float): 
            if not ("전국 다 가능" in list_user_region):
                list_activity_region = activity_region.split(", ")
                for a_r in list_activity_region :
                    if a_r in list_user_region : break
                else:
                    continue
        if (dt_activity['category'][j] not in list_user_category): continue # 해당 사용자가 원하는 카테고리에 없으면 아예 제외한다. 
        b_score = 0
        b_match = []
        i_score = 0
        i_match = []
        d_score = 0
        d_match = []
        p3_score = 0
        if dt_activity['b1'][j] =="o" and dt_user['b1'][i]=="O": b_score += 15; b_match.append("b1") 
        if dt_activity['b2'][j] =="o" and dt_user['b2'][i]=="O": b_score += 15; b_match.append("b2") 
        if dt_activity['b3'][j] =="o" and dt_user['b3'][i]=="O": b_score += 15; b_match.append("b3") 
        if dt_activity['b4'][j] =="o" and dt_user['b4'][i]=="O": b_score += 15; b_match.append("b4") 
        if dt_activity['b5'][j] =="o" and dt_user['b5'][i]=="O": b_score += 15; b_match.append("b5") 
        if dt_activity['b6'][j] =="o" and dt_user['b6'][i]=="O": b_score += 15; b_match.append("b6") 
        if dt_activity['b7'][j] =="o" and dt_user['b7'][i]=="O": b_score += 15; b_match.append("b7") 
        if dt_activity['b8'][j] =="o" and dt_user['b8'][i]=="O": b_score += 15; b_match.append("b8") 
        if dt_activity['b9'][j] =="o" and dt_user['b9'][i]=="O": b_score += 15; b_match.append("b9") 
        
        if dt_activity['i1'][j] in list_interest : i_score += 20; i_match.append(dt_activity['i1'][j])
        if dt_activity['i2'][j] in list_interest : i_score += 20; i_match.append(dt_activity['i2'][j])
        
        if dt_activity['d1'][j] in list_industry : d_score += 25; d_match.append(dt_activity['d1'][j])
        if dt_activity['d2'][j] in list_industry : d_score += 25; d_match.append(dt_activity['d2'][j])
        
        
        p3_score = b_score + i_score + d_score
        if len(b_match)==0 : b_match.append("-")
        if len(i_match)==0 : i_match.append("-")
        if len(d_match)==0 : d_match.append("-")
        dic_score[dt_activity['activity'][j]] = p3_score
        dic_benefit[dt_activity['activity'][j]] = b_match
        dic_interest[dt_activity['activity'][j]] = i_match
        dic_industry[dt_activity['activity'][j]] = d_match
    sorted_dic_score = sorted(dic_score.items(), key=operator.itemgetter(1), reverse=True)
    
    for n in range(0, 7):
        if(n<len(sorted_dic_score)):
            if(sorted_dic_score[n][1]>0):
                result.append(sorted_dic_score[n][0])
                result.append(sorted_dic_score[n][1])
                result.append(", ".join(dic_benefit[sorted_dic_score[n][0]]))
                result.append(", ".join(dic_interest[sorted_dic_score[n][0]]))
                result.append(", ".join(dic_industry[sorted_dic_score[n][0]]))
            else:
                result.append("-")
                result.append("-")
                result.append("-")
                result.append("-")
                result.append("-")
        else:
            result.append("-")
            result.append("-")
            result.append("-")
            result.append("-")
            result.append("-")
    return result

In [7]:
def score_of_per(p, dt_user):
    dt_result = DataFrame()
    dt_result = dt_result.append([["email", "category", "start", "end", "region",
                               "r1","r1_s","r1_b","r1_i","r1_d",
                               "r2","r2_s","r2_b","r2_i","r2_d",
                               "r3","r3_s","r3_b","r3_i","r3_d",
                               "r4","r4_s","r4_b","r4_i","r4_d",
                               "r5","r5_s","r5_b","r5_i","r5_d",
                               "r6","r6_s","r6_b","r6_i","r6_d",
                               "r7","r7_s","r7_b","r7_i","r7_d"]])
    
    for r in range(0, dt_user.index.size):    
        if(p==1):dt_result = dt_result.append([score_of_a_user_per1(r, dt_user)])
        elif(p==2):dt_result = dt_result.append([score_of_a_user_per2(r, dt_user)])
        elif(p==3):dt_result = dt_result.append([score_of_a_user_per3(r, dt_user)])
    return dt_result



In [8]:
writer = ExcelWriter('data/recommendation.xlsx')
rec_per1 = score_of_per(1, dt_user_per1) #"스펙과 무관한 다양한 경험, 새로운 사람들 만나보기."
rec_per2 = score_of_per(2, dt_user_per2) #"전공과 관련한 실무 경험과 교육, 역량 쌓기."
rec_per3 = score_of_per(3, dt_user_per3) #"취업에 도움되는 스펙 쌓기."

rec_per1.to_excel(writer, sheet_name='Per1', engine='xlsxwriter')
rec_per2.to_excel(writer, sheet_name='Per2', engine='xlsxwriter')
rec_per3.to_excel(writer, sheet_name='Per3', engine='xlsxwriter')
writer.save()